# Suspicious Outbound RDP Connections
Detects Non-Standard Tools Connecting to TCP port 3389 indicating possible lateral movement

## Rule Content
```
- title: Suspicious Outbound RDP Connections
  id: ed74fe75-7594-4b4b-ae38-e38e3fd2eb23
  status: experimental
  description: Detects Non-Standard Tools Connecting to TCP port 3389 indicating possible
    lateral movement
  references:
  - https://portal.msrc.microsoft.com/en-US/security-guidance/advisory/CVE-2019-0708
  author: Markus Neis - Swisscom
  date: 2019/05/15
  tags:
  - attack.lateral_movement
  - attack.t1210
  - car.2013-07-002
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 3
      DestinationPort: 3389
      Initiated: 'true'
    filter:
      Image:
      - '*\mstsc.exe'
      - '*\RTSApp.exe'
      - '*\RTS2App.exe'
      - '*\RDCMan.exe'
      - '*\ws_TunnelService.exe'
      - '*\RSSensor.exe'
      - '*\RemoteDesktopManagerFree.exe'
      - '*\RemoteDesktopManager.exe'
      - '*\RemoteDesktopManager64.exe'
      - '*\mRemoteNG.exe'
      - '*\mRemote.exe'
      - '*\Terminals.exe'
      - '*\spiceworks-finder.exe'
      - '*\FSDiscovery.exe'
      - '*\FSAssessment.exe'
      - '*\MobaRTE.exe'
      - '*\chrome.exe'
      - '*\thor.exe'
      - '*\thor64.exe'
    condition: selection and not filter
  falsepositives:
  - Other Remote Desktop RDP tools
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"3" AND dst_port:"3389" AND network_initiated:"true") AND (NOT (process_path.keyword:(*\\mstsc.exe OR *\\RTSApp.exe OR *\\RTS2App.exe OR *\\RDCMan.exe OR *\\ws_TunnelService.exe OR *\\RSSensor.exe OR *\\RemoteDesktopManagerFree.exe OR *\\RemoteDesktopManager.exe OR *\\RemoteDesktopManager64.exe OR *\\mRemoteNG.exe OR *\\mRemote.exe OR *\\Terminals.exe OR *\\spiceworks\-finder.exe OR *\\FSDiscovery.exe OR *\\FSAssessment.exe OR *\\MobaRTE.exe OR *\\chrome.exe OR *\\thor.exe OR *\\thor64.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()